In [8]:
import numpy as np
from sklearn.utils import shuffle
#Test
X = np.array([[1,2],[3,4],[5,6],[7,8]])
print(X.shape)
y = np.array([0,1,2,3])
X , y = shuffle(X,y, random_state=1)

print(X,y)

(4, 2)
[[7 8]
 [5 6]
 [1 2]
 [3 4]] [3 2 0 1]


In [9]:
from time import sleep
from tqdm import tqdm
for i in tqdm(range(10)):
    sleep(3)

100%|████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:30<00:00,  3.00s/it]
